In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import zoo_of_odes
import ode_collapser

In [ ]:
# Parameters

ode_name = 'harmonic_oscillator_with_quadratic_drag'
ode_params = {
    'omega': 2*np.pi*3,
    'nu': 4.0
}
ode_initial_conditions = {
    'x0': 0.8,
    'v0': 15.0,
}

h = 0.01  # Grid resolution
rng_seed_data = 123  # Used for data generation
sigma = 0.1  # Noise level for data generation
N_samples = 10  # Number of datapoints to 'measure' from the 'true' curve

In [ ]:
# Get solution for the ODE
x_true_grid, t_grid = zoo_of_odes.get_solution(
    ode_name,
    params=ode_params,
    initial_conditions=ode_initial_conditions,
    t_start=0.0,
    t_end=1.0,
    h=h,
)

plt.plot(t_grid, x_true_grid, ls='-', color='tab:blue')

In [ ]:
rng_data = np.random.RandomState(rng_seed_data)  # Instantiate the RNG in the same cell that we will do all the calls.
N_grid = t_grid.shape[0]
idx_samples = rng_data.choice(N_grid, size=N_samples, replace=False)  # Choose which datapoints we will 'measure'
idx_samples = np.sort(idx_samples)
x_noise = rng_data.normal(scale=sigma, size=(N_samples,))  # Sample noise to be added to our datapoints.
t_samples = t_grid[idx_samples]  # Used only for plotting in this example
x_samples = x_true_grid[idx_samples] + x_noise  # Noisy datapoints
del rng_data  # Delete to prevent re-use of this RNG in the solution section.

# Plot the generated data
fig, ax = plt.subplots()
ax.plot(t_grid, x_true_grid, ls='-', marker='none', label='True solution')
ax.plot(t_samples, x_samples, ls='none', marker='o', alpha=0.7, label='Samples / measurements')
ax.set_xlim(left=t_grid[0], right=t_grid[-1])
ax.set_xlabel('t')
ax.set_ylabel('x(t)')
ax.legend()

plt.show()

In [ ]:
collapser_results = ode_collapser.collapse_to_solution(
    rhs=zoo_of_odes.get_rhs_func(ode_name, ode_params),
    h=h,
    t_start=0.0,
    t_end=1.0,
    idx_samples=idx_samples,
    x_samples=x_samples,
    show_progress=True,
)

# Print the headline result: how well did we fit the data?
print(f'Loss due to MSE of data: {collapser_results["log_scalars"][-1]["loss_data"]}')
print(f'Loss due to ODE violation: {collapser_results["log_scalars"][-1]["loss_ODE"]}')

# Plot the fitted x(t)
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(t_grid, x_true_grid, ls='-', marker='none', label='True solution')
ax.plot(t_samples, x_samples, ls='none', marker='o', alpha=0.7, label='Samples / measurements')
ax.plot(t_grid, collapser_results['x_solution_grid'], ls='--', marker='none', label='Optimization result solution')
ax.set_xlim(left=t_grid[0], right=t_grid[-1])
ax.set_xlabel('t')
ax.set_ylabel('x(t)')
ax.legend()

plt.show()

In [ ]:
# What happens if we do the same thing, but with the 'wrong' ODE passed to the collapser?
# This should still produces a very low loss_ODE, but a higher loss_data.
collapser_results = ode_collapser.collapse_to_solution(
    rhs=zoo_of_odes.get_rhs_func(ode_name, {'omega': 2.0*np.pi*3.0, 'nu': 0.0}),
    h=h,
    t_start=0.0,
    t_end=1.0,
    idx_samples=idx_samples,
    x_samples=x_samples,
    show_progress=True,
)

# Print the headline result: how well did we fit the data?
print(f'Loss due to MSE of data: {collapser_results["log_scalars"][-1]["loss_data"]}')
print(f'Loss due to ODE violation: {collapser_results["log_scalars"][-1]["loss_ODE"]}')

# Plot the fitted x(t)
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(t_grid, x_true_grid, ls='-', marker='none', label='True solution')
ax.plot(t_samples, x_samples, ls='none', marker='o', alpha=0.7, label='Samples / measurements')
ax.plot(t_grid, collapser_results['x_solution_grid'], ls='--', marker='none', label='Optimization result solution')
ax.set_xlim(left=t_grid[0], right=t_grid[-1])
ax.set_xlabel('t')
ax.set_ylabel('x(t)')
ax.legend()

plt.show()
fig.savefig('./badfit_qd_osc_sho.png', bbox_inches='tight')

In [ ]:
# What happens if we do the same thing, but with the 'wrong' ODE passed to the collapser?
# This should still produces a very low loss_ODE, but a higher loss_data.
collapser_results = ode_collapser.collapse_to_solution(
    rhs=zoo_of_odes.get_rhs_func('constant_acceleration', {'a': -2.0}),
    h=h,
    t_start=0.0,
    t_end=1.0,
    idx_samples=idx_samples,
    x_samples=x_samples,
    show_progress=True,
)

# Print the headline result: how well did we fit the data?
print(f'Loss due to MSE of data: {collapser_results["log_scalars"][-1]["loss_data"]}')
print(f'Loss due to ODE violation: {collapser_results["log_scalars"][-1]["loss_ODE"]}')

# Plot the fitted x(t)
fig, ax = plt.subplots()
ax.plot(t_grid, x_true_grid, ls='-', marker='none', label='True solution')
ax.plot(t_samples, x_samples, ls='none', marker='o', alpha=0.7, label='Samples / measurements')
ax.plot(t_grid, collapser_results['x_solution_grid'], ls='--', marker='none', label='Optimization result solution')
ax.set_xlim(left=t_grid[0], right=t_grid[-1])
ax.set_xlabel('t')
ax.set_ylabel('x(t)')
ax.legend()

plt.show()